In [1]:
import pandas as pd
import numpy as np

SALES = pd.read_csv('lv2-2411.csv')
print(SALES.shape)
print(SALES.info())

(2295, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2295 entries, 0 to 2294
Data columns (total 12 columns):
SID      2295 non-null int64
DT       2295 non-null object
SALES    2295 non-null int64
HOLI     2295 non-null int64
MD1      2295 non-null int64
MD2      2295 non-null int64
MD3      2295 non-null int64
TEMP     2295 non-null float64
FUEL     2295 non-null float64
CPI      2295 non-null float64
UNEMP    2295 non-null float64
TT       2295 non-null object
dtypes: float64(4), int64(6), object(2)
memory usage: 215.3+ KB
None


### Question 1

In [2]:
# 1-1
DF21 = SALES.copy()
cor_abs = DF21[['SALES','MD1','MD2','MD3','TEMP','FUEL','CPI','UNEMP']].corr().iloc[0,1:].abs()
cor_abs

MD1      0.323809
MD2      0.227710
MD3      0.305563
TEMP     0.079040
FUEL     0.035965
CPI      0.071120
UNEMP    0.104058
Name: SALES, dtype: float64

In [3]:
# 1-2
cor_abs.max()

0.32380918221245447

In [4]:
# Answer 2
ans21f = cor_abs.max()
ans21 = round(ans21f, 3)

print(ans21f)
print(ans21)

0.32380918221245447
0.324


### Question 2

In [5]:
# 2-1
DF22 = SALES[SALES['DT']=='2024-04-22'].copy().reset_index(drop=True)
print(DF22.shape)

(45, 12)


In [6]:
# 2-2
from sklearn.preprocessing import StandardScaler
svar = ['SALES','MD1','MD2','MD3','TEMP','FUEL','CPI','UNEMP']
svars = [f"{x}_S" for x in svar]
scaler = StandardScaler()
DF22s = scaler.fit_transform(DF22[svar])
DF22s = pd.DataFrame(DF22s, columns = svars)
DF22f = pd.concat([DF22, DF22s], axis = 1)
DF22f.head()

,SID,DT,SALES,HOLI,MD1,MD2,MD3,TEMP,FUEL,CPI,UNEMP,TT,SALES_S,MD1_S,MD2_S,MD3_S,TEMP_S,FUEL_S,CPI_S,UNEMP_S
0,1,2024-04-22,1493660,0,2586,1057,1305,69.16,3.506,223.444251,6.573,TRAIN,0.925886,-0.573575,0.748461,-0.322387,0.781573,-1.032914,1.134592,-0.239531
1,2,2024-04-22,1834458,0,6153,272,2924,69.79,3.506,223.078337,6.170,TEST,1.581120,0.410093,-0.527571,0.686657,0.842296,-1.032914,1.125720,-0.493086
2,3,2024-04-22,405433,0,371,6,180,74.66,3.506,226.987364,6.034,TRAIN,-1.166390,-1.184403,-0.959958,-1.023546,1.311689,-1.032914,1.220499,-0.578653
3,4,2024-04-22,2149594,0,1763,0,7577,63.64,3.514,131.193097,3.879,TRAIN,2.187015,-0.800533,-0.969712,3.586649,0.249529,-1.003970,-1.102141,-1.934511
4,5,2024-04-22,319551,0,1548,133,599,71.70,3.506,224.037814,5.422,TRAIN,-1.331510,-0.859823,-0.753518,-0.762403,1.026391,-1.032914,1.148983,-0.963704


In [7]:
# 2-3
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 4, random_state = 1234, init = 'random').fit(DF22f[svars])
DF22f['clst'] = km.labels_

In [8]:
# 2-4
DF22out = DF22f.groupby(['clst']).mean()[['SALES','UNEMP']].reset_index()
unemp_maxidx = DF22out.SALES.idxmax()
DF22out

,clst,SALES,UNEMP
0,0,1.461851e+06,5.819125
1,1,1.589292e+06,8.201875
2,2,5.000801e+05,6.124875
3,3,8.159207e+05,7.226190


In [9]:
# Answer 2
ans22f = DF22out.iloc[unemp_maxidx]['UNEMP']
ans22 = round(ans22f, 3)

print(ans22f)
print(ans22)

8.201875
8.202


### Question 3

In [10]:
# 3-1
DF23 = SALES.copy()
DF23['SEASON1'] = 1*DF23['DT'].str[5:7].isin(['03','04','05'])
DF23['SEASON2'] = 1*DF23['DT'].str[5:7].isin(['06','07','08'])
DF23['SEASON3'] = 1*DF23['DT'].str[5:7].isin(['09','10','11'])

In [11]:
# 3-2
TRAIN23 = DF23[DF23['TT']=='TRAIN'].copy()
TEST23 = DF23[DF23['TT']=='TEST'].copy()
print(TRAIN23.shape, TEST23.shape)

(1722, 15) (573, 15)


In [12]:
# 3-3
from sklearn.linear_model import LinearRegression
FEAT23 = ['HOLI','MD1','MD2','MD3','TEMP','FUEL','CPI','UNEMP','SEASON1','SEASON2','SEASON3']
X_train = TRAIN23[FEAT23]
y_train = TRAIN23['SALES']
MDL23 = LinearRegression()
MDL23.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
# 3-4
X_test = TEST23[FEAT23]
y_pred = MDL23.predict(X_test)

MAPE = np.mean(np.abs((y_pred - TEST23['SALES'])/TEST23['SALES']))*100
print(MAPE)

53.59237851260662


In [14]:
# Answer 3
ans23f = MAPE
ans23 = round(ans23f, 2)

print(ans23f)
print(ans23)

53.59237851260662
53.59


## Summary

In [15]:
print(ans21)
print(ans22)
print(ans23)

0.324
8.202
53.59
